<a href="https://colab.research.google.com/github/vtu26220-boop/AIT-LABTASK-3/blob/main/AIT_LAP_26220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
state = {
    'monkey_location': 'door',
    'box_location': 'window',
    'monkey_on_box': False,
    'monkey_has_banana': False
}

goal_stack = ['has(monkey, banana)']
actions = []

def walk(x, y):
    state['monkey_location'] = y
    actions.append(f"Walk from {x} to {y}")

def push(x, y):
    state['monkey_location'] = y
    state['box_location'] = y
    actions.append(f"Push box from {x} to {y}")

def climb_up():
    state['monkey_on_box'] = True
    actions.append("Climb up the box")

def grasp():
    state['monkey_has_banana'] = True
    actions.append("Grasp the banana")

while goal_stack:
    goal = goal_stack.pop()

    if goal == 'has(monkey, banana)':
        if not state['monkey_has_banana']:
            goal_stack.append('perform(grasp)')
            goal_stack.append('at(monkey, middle)')
            goal_stack.append('on(monkey, box)')

    elif goal == 'perform(grasp)':
        if state['monkey_location'] == 'middle' and state['monkey_on_box']:
            grasp()

    elif goal == 'on(monkey, box)':
        if not state['monkey_on_box']:
            if state['monkey_location'] == state['box_location']:
                goal_stack.append('perform(climb_up)')
            else:
                goal_stack.append(f'at(monkey, {state["box_location"]})')

    elif goal == 'perform(climb_up)':
        climb_up()

    elif goal.startswith('at(monkey,'):
        target = goal.split(',')[1].strip(' )')
        if state['monkey_location'] != target:
            if state['monkey_location'] == state['box_location']:
                push(state['box_location'], target)
            else:
                walk(state['monkey_location'], target)

print("Monkey-Banana Planning Actions:")
for step in actions:
    print("▶", step)

print("\nFinal State:")
for key, value in state.items():
    print(f"{key}: {value}")


Monkey-Banana Planning Actions:
▶ Walk from door to window
▶ Push box from window to middle

Final State:
monkey_location: middle
box_location: middle
monkey_on_box: False
monkey_has_banana: False


task 7

In [2]:
import random
import numpy as np

# Distance matrix (simulated travel time between delivery points)
distance_matrix = [
    [0, 2, 2, 5, 7],
    [2, 0, 4, 8, 2],
    [2, 4, 0, 1, 3],
    [5, 8, 1, 0, 2],
    [7, 2, 3, 2, 0]
]

num_ants = 5
num_cities = len(distance_matrix)
num_iterations = 100
evaporation_rate = 0.5
alpha = 1  # pheromone importance
beta = 2   # distance importance

# Initialize pheromone levels
pheromone = [[1 for _ in range(num_cities)] for _ in range(num_cities)]

def calculate_probability(from_city, to_city, visited):
    if to_city in visited:
        return 0
    pher = pheromone[from_city][to_city] ** alpha
    heuristic = (1 / distance_matrix[from_city][to_city]) ** beta
    return pher * heuristic

def select_next_city(current_city, visited):
    probabilities = [calculate_probability(current_city, i, visited) for i in range(num_cities)]
    total = sum(probabilities)
    if total == 0:
        return random.choice([i for i in range(num_cities) if i not in visited])
    probabilities = [p / total for p in probabilities]
    return np.random.choice(range(num_cities), p=probabilities)

def build_solution():
    solutions = []
    for _ in range(num_ants):
        visited = []
        current_city = random.randint(0, num_cities - 1)
        visited.append(current_city)

        while len(visited) < num_cities:
            next_city = select_next_city(current_city, visited)
            visited.append(next_city)
            current_city = next_city
        solutions.append(visited)
    return solutions

def route_distance(route):
    return sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1)) + distance_matrix[route[-1]][route[0]]

def update_pheromones(solutions):
    global pheromone
    for i in range(num_cities):
        for j in range(num_cities):
            pheromone[i][j] *= (1 - evaporation_rate)
    for route in solutions:
        dist = route_distance(route)
        for i in range(num_cities):
            from_city = route[i]
            to_city = route[(i + 1) % num_cities]
            pheromone[from_city][to_city] += 1.0 / dist

best_route = None
best_distance = float('inf')

for _ in range(num_iterations):
    solutions = build_solution()
    update_pheromones(solutions)
    for route in solutions:
        dist = route_distance(route)
        if dist < best_distance:
            best_distance = dist
            best_route = route

print("Best route found:", best_route)
print("Total trip duration:", best_distance)


Best route found: [4, np.int64(3), np.int64(2), np.int64(0), np.int64(1)]
Total trip duration: 9


task 5

In [14]:
def print_board(board):
    for row in board:
        print(row)
    print()

def check_winner(board):
    for i in range(3):
        # Check rows
        if board[i][0] == board[i][1] == board[i][2] != '_':
            return board[i][0]
        # Check columns
        if board[0][i] == board[1][i] == board[2][i] != '_':
            return board[0][i]
    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] != '_':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != '_':
        return board[0][2]
    return None

def is_full(board):
    for row in board:
        if '_' in row:
            return False
    return True

def minimax(board, is_maximizing):
    winner = check_winner(board)
    if winner == 'X':
        return 1
    elif winner == 'O':
        return -1
    elif is_full(board):
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == '_':
                    board[i][j] = 'X'
                    score = minimax(board, False)
                    board[i][j] = '_'
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == '_':
                    board[i][j] = 'O'
                    score = minimax(board, True)
                    board[i][j] = '_'
                    best_score = min(score, best_score)
        return best_score

def best_move(board):
    best_score = -float('inf')
    move = None
    for i in range(3):
        for j in range(3):
            if board[i][j] == '_':
                board[i][j] = 'X'
                score = minimax(board, False)
                board[i][j] = '_'
                if score > best_score:
                    best_score = score
                    move = (i, j)
    return move


task 4